# Wegaw API Calls

This notebook contains examples to call the Wegaw API. To run it, you need to provide the credentials (`YOUR USERNAME` and `YOUR PASSWORD`) in the following cell. Once the credentials are provided, you can run the other cells with the default configuration.

Throughout this notebook, there are variables in capital letters - these are variables that you can customize for your needs.

The most up-to-date documentation, with examples on the API use in different programming languages (Python, JavaScript and cURL), can be found at https://api.staging.defrost.io/.

In [ ]:
## Installation Instructions

# To ensure the necessary libraries are installed, run the following cell:

%pip install requests
%pip install pandas

In [ ]:
base_url = 'https://api.staging.defrost.io'

In [ ]:
# Enter the username and password provided by Wegaw here

USERNAME = '<USERNAME>'
PASSWORD = '<PASSWORD>'

## Get Token Access

Token access is required to identify the user in the API calls. The token expires within 5 minutes of creation, and if an out-of-date token is used, a `401 error` will be returned. Therefore, the following cell must be rerun to obtain a new token when needed.

In [ ]:
import requests

url = f'{base_url}/v2/token/'
payload = {'username':USERNAME,'password':PASSWORD}
r = requests.post(url, json=payload, timeout=300)

if r.status_code == 200:
     data = r.json()
     TOKEN = data['access']
else:
    print(f'Failed with status code: {r.status_code}')

## Check Available Catchments

Here you can check for which catchments you can access data. The following example returns a GeoJSON for each available catchment.

In [ ]:
import requests

url = f'{base_url}/v2/catchments/'
headers = {'Authorization': 'Bearer ' + TOKEN}
r = requests.get(url, headers=headers, timeout=300)

if r.status_code == 200:
     data = r.json()
     CATCHMENT = data['features'][0]['properties']['name'] if len(data['features']) > 0 else print('No catchment available') # selecting, by default, the first catchment available
     for catchment in data['features']:
          print(f"Catchment: {catchment['properties']['display_name']} -> API catchemnt name: {catchment['properties']['name']}")
elif r.status_code == 401:
     print(f"Error {r.status_code}. It's possible that the access token has expired.")
else:
    print(f'Failed with status code: {r.status_code}')

## Access Products

This section shows how to retrieve product information from the API. First we will look at retrieving individual values, then rasters. You can define the product and the catchment (by default, it is set to the first catchment available in the previous cell call). The possible products are:
- 'swe' : Snow Water Equivalent
- 'sce' : Snow Cover Extent
- 'hs' : Snow Height
- 'swc' : Soil Water Content
- 'ms' : Melting Snow

In [ ]:
PRODUCT = 'swe' #'sce' 'hs' 'swc' 'ms' # name of the product in lowercase 
# CATCHMENT = '<CATCHMENT>' # here you can select a different catchment. To use this line of code, remove the # at the start.

### Individual Values

This section shows how to retrieve values for the selected product in the selected catchment. Values are returned in the format {value, date format}. 

Example ``{'value': 6.96, 'date': '2024-05-01'}``.

#### Show Last Value Available

Using the *latest* endpoint, the last available for the specified catchment/product is returned with its date.

In [ ]:
import requests
import pandas as pd

url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-stats/latest/'
headers = {'Authorization': 'Bearer ' + TOKEN}
r = requests.get(url, headers=headers, timeout=300)

if r.status_code == 200:
  data = r.json()
  df = pd.DataFrame(data, index=[0])
  print(df)
elif r.status_code == 204:
  print(f'No data available for catchment {CATCHMENT} and product {PRODUCT}')
elif r.status_code == 401:
     print(f"Error {r.status_code}. It's possible that the access token has expired.")
else:
    print(f'Failed with status code: {r.status_code}')

#### Show Available Values for a Certain Period of Time

This cell shows all available values for a certain period of time. By default, the last 30 days of values will be returned by the call, but the dates can be modified by specifing the ```FROM_DATE``` and ```TO_DATE``` parameters.

In [ ]:
import requests
import pandas as pd

url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-stats/'

# In case other dates are needed, remove the # at the start of each line of code.

# FROM_DATE = '2022-01-01'
# TO_DATE = '2022-01-07'
# url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-stats/?from={FROM_DATE}&to={TO_DATE}'

headers = {'Authorization': 'Bearer ' + TOKEN}
r = requests.get(url, headers=headers, timeout=300)
if r.status_code == 200:
  data = r.json()
  df = pd.DataFrame(data)
  print(df.tail())
elif r.status_code == 401:
     print(f"Error {r.status_code}. It's possible that the access token has expired.")
else:
  print(f'Failed with status code: {r.status_code}')

#### Show Available Values in mm (this only affects the swe and hs products)

By using the *units* parameter in the URL, the swe or hs values can be retrieved in millimeters (mm). By default, the values for swe and hs are provided in centimeters (cm).

In [ ]:
import requests
import pandas as pd

url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-stats/latest?units=mm'
headers = {'Authorization': 'Bearer ' + TOKEN}
r = requests.get(url, headers=headers, timeout=300)

if r.status_code == 200:
  data = r.json()
  df = pd.DataFrame(data, index=[0])
  print(df)
elif r.status_code == 204:
  print(f'No data available for catchment {CATCHMENT} and product {PRODUCT}')
elif r.status_code == 401:
     print(f"Error {r.status_code}. It's possible that the access token has expired.")                      
else:
    print(f'Failed with status code: {r.status_code}')

In [ ]:
import requests
import pandas as pd

url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-stats?units=mm'

# In case other dates are needed, remove the # at the start of each line of code.

# FROM_DATE = '2022-01-01'
# TO_DATE = '2022-01-07'
# url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-stats/?from={FROM_DATE}&to={TO_DATE}'

headers = {'Authorization': 'Bearer ' + TOKEN}
r = requests.get(url, headers=headers, timeout=300)
if r.status_code == 200:
  data = r.json()
  df = pd.DataFrame(data)
  print(df.tail())
elif r.status_code == 401:
     print(f"Error {r.status_code}. It's possible that the access token has expired.")
else:
  print(f'Failed with status code: {r.status_code}')

### Rasters

This section shows how to retrieve information about the availability of rasters for each catchment and product, and how to download available raster files.

#### Show All Avaialble Rasters for a Certain Period of Time

This cell shows the available rasters  for a certain period of time. By default, the last 30 days of values will be returned by the call, but the dates can be modified by specifing the ```FROM_DATE``` and ```TO_DATE``` parameters.

In [ ]:
import requests
import pandas as pd

url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-rasters/'

# In case other dates are needed, remove the # at the start of each line of code.

# FROM_DATE = '2022-01-01'
# TO_DATE = '2022-01-07'

# url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-rasters/?from={FROM_DATE}&to={TO_DATE}'

headers = {'Authorization': 'Bearer ' + TOKEN}
r = requests.get(url, headers=headers, timeout=300)
if r.status_code == 200:
  data = r.json()
  FILENAME =  data['snowmaps'][-1]['file_name'] if len(data['snowmaps']) > 0 else ''# Taking the last filename by default
  df = pd.DataFrame(data['snowmaps'])
  print(df.tail())
else:
  print(f'Failed with status code: {r.status_code}')

#### Download Rasters

By default, the file downloaded will be the first of the filenames returned in the cell above. You can change this by modifying the ``FILENAME`` parameter. By default, the downloaded file will be saved with the same filename, in the current working directory. You can change this by modifying the ``OUTPUT`` parameter. 

In [ ]:
import urllib.request

# FILENAME = '<FILENAME>' # Modify this if you want to download a file other than the first one from the filenames returned in the cell above. Remove the # at the start to run the line of code.
OUTPUT = FILENAME # Modify this if you want to specify a different filename/directory than the original filename, e.g. 'mycatchment_swe_2024_04_02.tif'

url = f'{base_url}/v2/catchments/{CATCHMENT}/{PRODUCT}-rasters/{FILENAME}'  # file_name from the API call
opener = urllib.request.build_opener()
opener.addheaders = [('Authorization', 'Bearer ' + TOKEN)]
urllib.request.install_opener(opener)
try:
    response = urllib.request.urlopen(url)
    if response.getcode() == 200:
        urllib.request.urlretrieve(url, OUTPUT)
        print(f'File downloaded successfully to {OUTPUT}.')
except Exception as e:
    print(f'Failed to download the file. Response code: {e}')